Laad nuttige functionaliteit

In [ ]:
# Creeren van figuren 
using Plots, LaTeXStrings
plot_options = (legend=false,linewidth=5)

# Gebruik van widget met slider (als het toeval het toestaat)
using Interact 

# Verzameling van bases 
using BasisFunctions 

Basisfunctions.jl is een pakket van onze onderzoeksgroep en bevat functionaliteit voor het benaderen van een functie met een verzameling van gegeven functies

# Bases met veeltermen voor benadering op een interval

## 1. Monomialen

We kunnen functies benaderen met bijvoorbeeld een set in de vorm 

$$ 1, x, x^2, x^3, \dots $$

We benaderen dus $f(x)$ met 
$$ c_0 + c_1x+c_2x^2 +\cdots + c_Nx^N  $$
Of in het kort 
$$ f(x)\approx \sum_{k=0}^N c_kx^k$$

De onbekenden zijn 
$$ c_0,\quad c_1,\quad c_2,\quad \cdots,\quad c_N$$

en kunnen gevonden worden door het oplossen van een vierkante $N\times N$matrix 


$$\begin{bmatrix}  
1 & x_1 & x_1^2 & \cdots & x_1^N \\1 & x_2 & x_2^2 & \cdots & x_2^N \\\vdots & \vdots & \vdots & \ddots & \vdots \\1 & x_N & x_N^2 & \cdots & x_N^N \\\end{bmatrix} \begin{bmatrix}  c_0 \\c_1 \\\vdots \\c_N \\\end{bmatrix} = \begin{bmatrix} f(x_1) \\ f(x_2) \\ \vdots \\f(x_N)\end{bmatrix} $$

In [ ]:
N = 10
basis = Monomials(N+1)
gr();@manipulate for i in 1:length(basis) 
    plot(x->basis[i](x), -1,1;ylims=(-1,1),plot_options...,title=latexstring("x^$(i-1)"))
end

Dit is echter een <i><b>onstabiele</b></u> basis omdat de verschillende functies goed op elkaar beginnen lijken voor hoge graad.  

In [ ]:
basis = Monomials(N+1)
gr();plot(x->basis[end](x), -1,1;ylims=(0,1),plot_options...)
plot!(x->basis[end-2](x), -1,1;ylims=(0,1),plot_options...)

## 1. Chebyshev

Een chebyshev basis bevat ook veeltermen, maar is stabieler

$$ T_0(x) = 1, \quad T_1(x) = x,\quad T_2(x) = 2x^2-1, \quad T_3(x) = 4x^3-3x,\quad\dots $$ 

In [ ]:
basis = ChebyshevT(N);
gr();@manipulate for k in 1:length(basis)
    plot(x->basis[k](x), -1,1;ylims=(-1,1),plot_options...,title=latexstring("T_$(k-1)(x)"))
end

Stel dat we een exponentiele functie $f(x)=e^x$ willen benaderen in $[-1,1]$

In [ ]:
line_fun = exp
plot(line_fun,-1,1,legend=false,linewidth=10)

   $$ e^x≈\sum_{k=0}^N c_kT_k(x) $$

In [ ]:
gr();@manipulate for n in 1:21
    fun = approximate(ChebyshevT(n),line_fun)
    plot(fun;layout=3,plot_options...,title="Benadering")
    plot!(line_fun,-1,1;subplot=1,color="black")
    plot!(fun,line_fun;subplot=2,ylims=(1e-16,1),plot_options...,title="Fout")
    scatter!(abs.(coefficients(fun));ylims=(1e-18,10),yscale=:log10,subplot=3,title=L"$c_k$",plot_options...)
end

# Fourier basis voor benaderen op een cirkel

Een Fourier basis bevat cosinussen en sinussen: 
$$ 1, \quad\cos(\pi x), \quad \sin (\pi x),\quad \cos(2\pi x),\quad \sin (2\pi x), \dots$$

In [ ]:
ϕ= basis[1]

In [ ]:
ϕ.index

In [ ]:
basis = Fourier(N+1);
gr();@manipulate for i in 1:length(basis) 
    plot(basis[i];ylims=(-1,1),plot_complex=true,plot_options...)
end

Deze basis is periodic en is bijgevolg goed om functies op een cirkel te benaderen 

In [ ]:
θs = LinRange(0,1,500)
zs = cis.(2pi.*θs)
xs = real.(zs)
ys = imag.(zs)
gr();plot3d(xs,ys,zeros(length(θs));plot_options...)

In [ ]:
cirkel_f = θ->exp(cos(6π*θ))-.7
zs = cirkel_f.(θs)
gr();plot3d(xs,ys,zs;color=cgrad(:redsblues)[zs],plot_options...,
        xlabel="x",ylabel="y",zlabel="z")

In [ ]:
gr();@manipulate throttle = 0.1  for n in 1:2:101
    fun = approximate(Fourier(n),cirkel_f)
    zs = real.(fun.(θs))
    plot3d(xs,ys,zs;color=cgrad(:redsblues)[zs],layout=3,plot_options...,title="Benadering")
    plot3d!(xs,ys,cirkel_f.(θs);subplot=1,color="black")
    plot!(fun,cirkel_f;ylims=(1e-16,1),subplot=2,title="Fout",xlabel="θ",ylabel="z")
    scatter!(abs.(coefficients(fun));ylims=(1e-18,10),yscale=:log10,subplot=3,title=L"$c_k$",xlabel="k",ylabel="",plot_options...)
end